# FFT Numba MPI

- https://numba.pydata.org/numba-doc/latest/user/withobjmode.html

In [11]:
%%writefile bc2cp.py
import numpy as np, time as tm
from numba import njit, objmode
from mpi4py_fft import PFFT, newDistArray
from mpi4py import MPI

t3 = tm.time()    # time measurement

def uu() : 
    return newDistArray(f, False)
def uf(u) :
    return f.forward(u, normalize=False)    
@njit
def ff() :
    with objmode(t0 = 'f8') :
        t0 = tm.time()    # time measurement
    
    # data
    with objmode(u = 'complex128[:,:,:]') :  # annotate return type
        u = uu()
    for k in range (u.shape[2]) :
        for j in range(u.shape[1]) :
            for i in range(u.shape[0]) :
                u[i, j, k] = complex( np.sin ( i + j + k + 3 ), 0 )

    with objmode(t1 = 'f8') :
        t1 = tm.time()    # time measurement

    # FFT
    with objmode(u_hat = 'complex128[:,:,:]') :  # annotate return type
        u_hat = uf(u)
    # checksum
    s = np.array(np.sum(u_hat), dtype=np.complex128)
    rs = np.array(0, dtype=np.complex128)
    with objmode() :
        MPI.COMM_WORLD.Reduce([s, MPI.DOUBLE_COMPLEX],
            [rs, MPI.DOUBLE_COMPLEX], op=MPI.SUM, root=0)

    with objmode(t2 = 'f8') :
        t2 = tm.time()    # time measurement

    return rs, t0, t1, t2

# main
ms = MPI.COMM_WORLD.Get_size()
mr = MPI.COMM_WORLD.Get_rank()
L = M = N = 576
# PFFT should be outside the numba function due to the "class" return
f = PFFT(MPI.COMM_WORLD, [N, M, L], dtype=np.complex128,
         backend='pyfftw')
# numba function
s, t0, t1, t2 = ff()

t4 = tm.time()    # time measurement

if not mr :
    print(f"S:{s*1E-5:.0f}", end='')
    print(f", L:{L:0g}", end='')
    print(f", N:{ms:0g}", end='')
    print(f", T1:{t1-t0:.4f}", end='')
    print(f", TF:{t2-t1:.4f}", end='')
    print(f", TT:{t2-t0:.4f}", end='')
    print(f", TO:{t4-t3:.4f}")

Overwriting bc2cp.py


## copy to /scratch

In [23]:
%%bash
dir=/scratch${PWD#"/prj"}
rm -f $dir/bc2cp.py
cp bc2cp.py $dir/

In [24]:
%%writefile bc2cp.srm
#!/bin/bash
#SBATCH --job-name bc2cp       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:02:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd
dir=/scratch${PWD#"/prj"}
cd $dir
source $dir/env2/etc/profile.d/conda.sh
conda activate $dir/env2
conda activate --stack $dir/env3
cd $dir/fft
                                              
# Executable
EXEC="python bc2cp.py"

# Start
echo '-- run --------------------------------'
echo '$ srun  --mpi=pmi2  -n' $SLURM_NTASKS  ${EXEC##*/}
echo '-- output -----------------------------'
srun  --mpi=pmi2  -n $SLURM_NTASKS  $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting bc2cp.srm


<hr style="height:10px;border-width:0;background-color:green">

## Run

### 1 of (1, 4, 16, 24, 48, 72, 96)

In [29]:
! sbatch --ntasks=1 bc2cp.srm
! sbatch --ntasks=1 bc2cp.srm
! sbatch --ntasks=1 bc2cp.srm

Submitted batch job 1334404
Submitted batch job 1334405
Submitted batch job 1334406


In [2]:
! cat /scratch${PWD#"/prj"}/slurm-1334404.out
! cat /scratch${PWD#"/prj"}/slurm-1334405.out
! cat /scratch${PWD#"/prj"}/slurm-1334406.out

- Job ID: 1334404
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 1 python bc2cp.py
-- output -----------------------------
S:270-0j, L:576, N:1, T1:21.6676, TF:19.2685, TT:40.9361, TO:52.5499
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334405
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 1 python bc2cp.py
-- output -----------------------------
S:270-0j, L:576, N:1, T1:21.6530, TF:19.1665, TT:40.8195, TO:48.2080
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334406
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft


<hr style="height:10px;border-width:0;background-color:green">

### 4 of (1, 4, 16, 24, 48, 72, 96)

In [30]:
! sbatch --ntasks=4 bc2cp.srm
! sbatch --ntasks=4 bc2cp.srm
! sbatch --ntasks=4 bc2cp.srm

Submitted batch job 1334407
Submitted batch job 1334408
Submitted batch job 1334409


In [3]:
! cat /scratch${PWD#"/prj"}/slurm-1334407.out
! cat /scratch${PWD#"/prj"}/slurm-1334408.out
! cat /scratch${PWD#"/prj"}/slurm-1334409.out

- Job ID: 1334407
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 4 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:4, T1:5.7041, TF:4.1165, TT:9.8206, TO:13.4497
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334408
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 4 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:4, T1:5.6955, TF:4.1385, TT:9.8340, TO:13.4932
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334409
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List

<hr style="height:10px;border-width:0;background-color:green">

### 16 of (1, 4, 16, 24, 48, 72, 96)

In [31]:
! sbatch --ntasks=16 bc2cp.srm
! sbatch --ntasks=16 bc2cp.srm
! sbatch --ntasks=16 bc2cp.srm

Submitted batch job 1334410
Submitted batch job 1334411
Submitted batch job 1334412


In [4]:
! cat /scratch${PWD#"/prj"}/slurm-1334410.out
! cat /scratch${PWD#"/prj"}/slurm-1334411.out
! cat /scratch${PWD#"/prj"}/slurm-1334412.out

- Job ID: 1334410
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 16 python bc2cp.py
-- output -----------------------------
S:270-0j, L:576, N:16, T1:1.5066, TF:1.5461, TT:3.0527, TO:5.0028
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334411
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 16 python bc2cp.py
-- output -----------------------------
S:270-0j, L:576, N:16, T1:1.5504, TF:1.3927, TT:2.9431, TO:5.6229
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334412
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
-

<hr style="height:10px;border-width:0;background-color:green">

### 24 of (1, 4, 16, 24, 48, 72, 96)

In [32]:
! sbatch --ntasks=24 bc2cp.srm
! sbatch --ntasks=24 bc2cp.srm
! sbatch --ntasks=24 bc2cp.srm

Submitted batch job 1334413
Submitted batch job 1334414
Submitted batch job 1334415


In [5]:
! cat /scratch${PWD#"/prj"}/slurm-1334413.out
! cat /scratch${PWD#"/prj"}/slurm-1334414.out
! cat /scratch${PWD#"/prj"}/slurm-1334415.out

- Job ID: 1334413
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 24 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:24, T1:0.9384, TF:1.0972, TT:2.0356, TO:4.0663
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334414
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 24 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:24, T1:0.9281, TF:1.0725, TT:2.0006, TO:4.1078
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334415
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
-

<hr style="height:10px;border-width:0;background-color:green">

### 48 of (1, 4, 16, 24, 48, 72, 96)

In [33]:
! sbatch --ntasks=48 bc2cp.srm
! sbatch --ntasks=48 bc2cp.srm
! sbatch --ntasks=48 bc2cp.srm

Submitted batch job 1334416
Submitted batch job 1334417
Submitted batch job 1334418


In [6]:
! cat /scratch${PWD#"/prj"}/slurm-1334416.out
! cat /scratch${PWD#"/prj"}/slurm-1334417.out
! cat /scratch${PWD#"/prj"}/slurm-1334418.out

- Job ID: 1334416
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1488 sdumont1489
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 48 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:48, T1:0.4478, TF:7.1442, TT:7.5920, TO:16.6838
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334417
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1488 sdumont1489
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 48 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:48, T1:0.3993, TF:7.2215, TT:7.6208, TO:9.6902
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334418
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which 

<hr style="height:10px;border-width:0;background-color:green">

### 72 of (1, 4, 16, 24, 48, 72, 96)

In [34]:
! sbatch --ntasks=72 bc2cp.srm
! sbatch --ntasks=72 bc2cp.srm
! sbatch --ntasks=72 bc2cp.srm

Submitted batch job 1334419
Submitted batch job 1334420
Submitted batch job 1334421


In [7]:
! cat /scratch${PWD#"/prj"}/slurm-1334419.out
! cat /scratch${PWD#"/prj"}/slurm-1334420.out
! cat /scratch${PWD#"/prj"}/slurm-1334421.out

- Job ID: 1334419
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 72
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1488 sdumont1489 sdumont1494
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 72 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:72, T1:0.1744, TF:8.3119, TT:8.4863, TO:10.2912
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334420
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 72
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1488 sdumont1489 sdumont1494
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 72 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:72, T1:0.1697, TF:9.7315, TT:9.9012, TO:11.7162
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334421
- Tasks per node:
- # of nodes in the job: 3
- # of ta

<hr style="height:10px;border-width:0;background-color:green">

### 96 of (1, 4, 16, 24, 48, 72, 96)

In [35]:
! sbatch --ntasks=96 bc2cp.srm
! sbatch --ntasks=96 bc2cp.srm
! sbatch --ntasks=96 bc2cp.srm

Submitted batch job 1334422
Submitted batch job 1334423
Submitted batch job 1334424


In [8]:
! cat /scratch${PWD#"/prj"}/slurm-1334422.out
! cat /scratch${PWD#"/prj"}/slurm-1334423.out
! cat /scratch${PWD#"/prj"}/slurm-1334424.out

- Job ID: 1334422
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1488 sdumont1489 sdumont1494 sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 96 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:96, T1:0.1321, TF:5.6489, TT:5.7809, TO:8.7118
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334423
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1488 sdumont1489 sdumont1494 sdumont1495
-- modules ----------------------------
-- run --------------------------------
$ srun  --mpi=pmi2  -n 96 python bc2cp.py
-- output -----------------------------
S:270+0j, L:576, N:96, T1:0.1245, TF:7.2284, TT:7.3530, TO:10.6695
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1334424
- Tasks per node:
- # of nodes 

<hr style="height:10px;border-width:0;background-color:red">

## Notes

In [22]:
! srun --mpi=list

srun: MPI types are...
srun: mpi/mpich1_p4
srun: mpi/mvapich
srun: mpi/none
srun: mpi/openmpi
srun: mpi/pmi2
srun: mpi/mpichgm
srun: mpi/lam
srun: mpi/mpich1_shmem
srun: mpi/mpichmx


* do not work: pmi2, openmpi, mpich1_p4, none, mpichmx